In [9]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\szbdr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
def readData(fname, startLine, endLine):
    count = 0
    for l in open(fname, encoding="UTF-8"):
        if count < startLine:
            count += 1
            continue
        elif count < endLine:
            l=l.replace(":null",'""')
            yield eval(l)
            count += 1
        else:
            break

In [3]:
review = list(readData("yelp_academic_dataset_review.json",0,100000))

In [30]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
stopwordSet = set()
#Process stopwords set
for sw in stopwords.words('english'):
    nonPunSW = ''.join([c for c in sw if not c in punctuation])
    stopwordSet.add(stemmer.stem(nonPunSW))

for d in review:
    r = ''.join([c for c in d['text'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        if w not in stopwordSet:
            wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [31]:
words = [x[1] for x in counts[:2000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [34]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        if w not in stopwordSet:
            if w in wordSet:
                feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

In [35]:
X = [feature(d) for d in review]
y = [d['stars'] for d in review]

In [36]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
mean_squared_error(y, predictions)

1.0015833127080562

In [37]:
valReview = list(readData("yelp_academic_dataset_review.json",100000,125000))

In [38]:
valX = [feature(d) for d in valReview]
valy = [d['stars'] for d in valReview]
valPredictions = clf.predict(valX)
mean_squared_error(valy, valPredictions)

1.084969028450438

In [39]:
clf10 = linear_model.Ridge(10.0, fit_intercept=False)
clf10.fit(X, y)
theta10 = clf10.coef_
predictions10 = clf10.predict(X)
mean_squared_error(y, predictions10)

1.0016029953269734

In [40]:
valPredictions10 = clf10.predict(valX)
mean_squared_error(valy, valPredictions10)

1.0840436830089752

In [41]:
clf100 = linear_model.Ridge(100.0, fit_intercept=False)
clf100.fit(X, y)
theta100 = clf100.coef_
predictions100 = clf100.predict(X)
mean_squared_error(y, predictions100)

1.0030684166624007

In [42]:
valPredictions100 = clf100.predict(valX)
mean_squared_error(valy, valPredictions100)

1.0776533427746924

(0.8012942179998677, 0.8014875160601806, 0.8038160520077358)